# CeSci cadCAD model

This notebook contains a cadCAD model of the centralized science value flow as shown in the schema below. This model shows the value flow within science, from funding to publishing, and the value leakages throughout. 

![](opsci_baseline_stock&flow.png)

## Questions

## Mathematical Specification

Funding that reaches researchers:

$F_r = F_i \cdot (1-\alpha)$

where $\alpha \in [0,1]$ is the management cost weight, $F_i$ is the initial funding disbursed by the funding agency.

Value of created research assets:

$V_r = F_r \cdot (1-\epsilon)-c_p$

Salary of researchers:

$S_r = V_r \cdot \beta$

where $\beta\in[0,1]$ is the percentage of the research value that originated due to the direct work of the researcher.

where $\epsilon>0$ is the leakage due to innefficient resource allocation.

Value that ends up in journals:

$V_j = V_r + c_b$

where $c_p$ is the cost of publishing, $c_b$ is the cost of buying access to knowledge resource (in case a researcher buys from the journal).

## cadCAD Standard Notebook Layout

### 0. Dependencies

In [1]:
import pandas as pd
import numpy as np
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor
from cadCAD import configs

### 1. State Variables

In [39]:
initial_state = {
    'funding_pool': 1000,
    'researcher_value': 0,
    'journal_value': 0,
    'losses': 0
}

### 2. System Parameters

In [65]:
system_params = {
    'funding_pool': [10000],
    'funding_round': [100],
    'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'beta': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'cost_publishing': [10],
}

In [52]:
system_params = {
    'funding_pool': [10000],
    'funding_round': [100],
    'alpha': [0.1],
    'beta': [0.5],
    'epsilon': [0.1],
    'cost_publishing': [10]
}

### 3. Policy Functions

Example:
Given initial funding 100, alpha 0.2, the funding that reaches the researchers is $100\cdot 0.8=80$. Next, given $\epsilon=0.1$, cost of publishing 10, the value of the research project is $80\cdot 0.9-10=62$.

In [53]:
def p_researher_funding(params, substep, state_history, previous_state):
    pool = params['funding_round']
    management_costs = pool * params['alpha']
    to_researcher = pool - management_costs
    return {'update_researcher_funding': to_researcher}

def p_journal_and_researcher(params, substep, state_history, previous_state):
    funding = p_researher_funding(params, substep, state_history, previous_state)
    research_value = funding['update_researcher_funding'] * (1-params['epsilon'])
    salary = research_value * params['beta']
    to_journal = research_value + params['cost_publishing']
    return {'update_journal': to_journal,
            'update_researcher_done': salary}

def p_funding_pool(params, substep, state_history, previous_state):
    funding = params['funding_round']
    return {'update_funding_pool': -funding}

def p_losses(params, substep, state_history, previous_state):
    value_start = params['funding_round']
    value_end = p_journal_and_researcher(params, substep, state_history, previous_state)
    losses = value_start - value_end['update_journal']
    return {'update_losses': losses}

### 4. State Update Functions

In [54]:
def s_researcher_value(params, substep, state_history, previous_state, policy_input):
    research_funding = policy_input['update_researcher_funding']
    research_salary = policy_input['update_researcher_done']
    research_value = previous_state['researcher_value']

    if research_salary == 0:
        updated_researcher_value = research_funding + research_value
        return 'researcher_value', updated_researcher_value
    else:
        updated_researcher_value = research_salary + research_value
        return 'researcher_value', updated_researcher_value

def s_journal_value(params, substep, state_history, previous_state, policy_input):
    to_journal = policy_input['update_journal']
    journal_value = previous_state['journal_value']

    updated_journal_value = to_journal + journal_value

    return 'journal_value', updated_journal_value

def s_funding_pool(params, substep, state_history, previous_state, policy_input):
    funding_pool = previous_state['funding_pool']
    updated_funding_pool = funding_pool - params['funding_round']
    return 'funding_pool', updated_funding_pool

def s_losses(params, substep, state_history, previous_state, policy_input):
    losses = previous_state['losses']
    updated_losses = losses + policy_input['update_losses']
    return 'losses', updated_losses

### 5. Partial State Update Blocks

In [55]:
partial_state_update_blocks = [
    {
        'policies': {
            'update_funding_pool': p_funding_pool,
            'update_research_funding': p_researher_funding,
            'update_journal_researcher_value': p_journal_and_researcher,
            'update_losses': p_losses
        },
        'variables': {
            'funding_pool': s_funding_pool,
            'researcher_value': s_researcher_value,
            'journal_value': s_journal_value,
            'losses': s_losses
        }
    }
]

### 6. Configuration

In [76]:
sim_config = config_sim({
    'N': 100,
    'T': range(10),
    'M': system_params
})

del configs[:]

In [77]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [68]:
print(system_params)

{'funding_pool': [10000], 'funding_round': [100], 'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'beta': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 'cost_publishing': [10]}


### 7. Execution

In [78]:
exec_context = ExecutionContext()
configs = experiment.configs
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 10, 6, 900, 4)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (10, 6, 900, 4)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 46.12s


### 8. Analysis

In [60]:
pd.options.plotting.backend = "plotly"

In [79]:
df = pd.DataFrame(raw_result)
df

,funding_pool,researcher_value,journal_value,losses,simulation,subset,run,substep,timestep
0,1000,0.0,0.0,0.0,0,0,1,0,0
1,900,8.1,91.0,9.0,0,0,1,1,1
2,800,16.2,182.0,18.0,0,0,1,1,2
3,700,24.3,273.0,27.0,0,0,1,1,3
4,600,32.4,364.0,36.0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...
9895,400,5.4,66.0,534.0,0,8,100,1,6
9896,300,6.3,77.0,623.0,0,8,100,1,7
9897,200,7.2,88.0,712.0,0,8,100,1,8
9898,100,8.1,99.0,801.0,0,8,100,1,9


In [80]:
df.plot(kind='line', x='timestep', y=['funding_pool','researcher_value', 'journal_value', 'losses'])

In [81]:
df.plot(kind='line', x='timestep', y=['funding_pool','researcher_value'])

In [82]:
df.plot(kind='line', x='timestep', y=['funding_pool', 'losses'])

In [83]:
df.plot(kind='line', x='timestep', y=['funding_pool', 'journal_value'])